<a href="https://colab.research.google.com/github/MehrdadJannesar/CNN_AdaBoost/blob/master/Jannesar_Shooshtari_Laitec_Improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np


# Import MNIST data
mnist = input_data.read_data_sets("../1-dnn/", one_hot=True)

# Training Parameters
num_steps = 200
batch_size = 128
display_step = 10
strides = 1
k = 2

# Network Parameters
num_input = 784  # MNIST data input (img shape: 28*28)
num_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.5  # Dropout, probability to keep unit

# tf Graph input
X = tf.placeholder('float', [None, num_input])
Y = tf.placeholder('float',shape=[None,num_classes])
keep_prob = tf.placeholder(tf.float32)  # dropout (keep probability)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../1-dnn/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../1-dnn/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ../1-dnn/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ../1-dnn/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tenso

In [0]:
def model_cnn(x,dropout,num_classes):
  x = tf.reshape(x, shape=[-1,28,28,1])

 
  w_c_1 = tf.Variable(tf.random_normal([3, 3, 1, 32]))
  w_c_2 = tf.Variable(tf.random_normal([3, 3, 32, 64]))
  w_c_3 = tf.Variable(tf.random_normal([3, 3, 64, 128]))
  b_c_1 = tf.Variable(tf.zeros([32]))
  b_c_2 = tf.Variable(tf.zeros([64]))
  b_c_3 = tf.Variable(tf.zeros([128]))
  # The second three convolutional layer weights
  w_c_4 = tf.Variable(tf.random_normal([3, 3, 128, 256]))
  w_c_5 = tf.Variable(tf.random_normal([3, 3, 256, 512]))
  w_c_6 = tf.Variable(tf.random_normal([3, 3, 512, 1024]))
  b_c_4 = tf.Variable(tf.zeros([256]))
  b_c_5 = tf.Variable(tf.zeros([512]))
  b_c_6 = tf.Variable(tf.zeros([1024]))
  # Fully connected weight
  w_f_1 = tf.Variable(tf.random_normal([7*7*1024, 2048]))
  w_f_2 = tf.Variable(tf.random_normal([2048, 1024]))
  w_f_3 = tf.Variable(tf.random_normal([1024, 512]))
  b_f_1 = tf.Variable(tf.zeros([2048]))
  b_f_2 = tf.Variable(tf.zeros([1024]))
  b_f_3 = tf.Variable(tf.zeros([512]))
  
  w_out = tf.Variable(tf.random_normal([512, num_classes]))
  b_out = tf.Variable(tf.zeros([num_classes]))
  
  # first layer convolution
  conv1 = tf.nn.bias_add(tf.nn.conv2d(x,w_c_1,name='C1',strides=[1,1,1,1],padding='SAME'),b_c_1)
  conv1 = tf.nn.relu(conv1)
  
  # second layer convolution
  conv2 = tf.nn.bias_add(tf.nn.conv2d(conv1,w_c_2,name='C2',strides=[1,1,1,1],padding='SAME'),b_c_2)
  conv2 = tf.nn.relu(conv2)
  
  # third layer convolution
  conv3 = tf.nn.bias_add(tf.nn.conv2d(conv2,w_c_3,name='C3',strides=[1,1,1,1],padding='SAME'),b_c_3)
  conv3 = tf.nn.relu(conv3)
  
  # first Max Pooling (down-sampling)
  pool_1 = tf.nn.max_pool2d(conv3,name='p1', ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
  
  # fourth layer convolution
  conv4 = tf.nn.bias_add(tf.nn.conv2d(pool_1,w_c_4,name='C4',strides=[1,1,1,1],padding='SAME'),b_c_4)
  conv4 = tf.nn.relu(conv4)
  
  # fifth layer convolution
  conv5 = tf.nn.bias_add(tf.nn.conv2d(conv4,w_c_5,name='C5',strides=[1,1,1,1],padding='SAME'),b_c_5)
  conv5 = tf.nn.relu(conv5)
  
  # sixth layer convolution
  conv6 = tf.nn.bias_add(tf.nn.conv2d(conv5,w_c_6,name='C6',strides=[1,1,1,1],padding='SAME'),b_c_6)
  conv6 = tf.nn.relu(conv6)
  
  # second Max Pooling (down-sampling)
  pool_2 = tf.nn.max_pool2d(conv6,name='p2',ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
  
  # first Fully connected layer
  # Reshape conv6 output to fit fully connected layer input
  # fc1 = tf.layers.dense(pool_2, 1024)
  fc1 = tf.reshape(pool_2, [-1, w_f_1.get_shape().as_list()[0]])
  fc1 = tf.add(tf.matmul(fc1, w_f_1), b_f_1)
  fc1 = tf.nn.relu(fc1)
  # Apply Dropout
  fc1 = tf.nn.dropout(fc1, dropout)
  
  # second Fully connected layer
  # fc2 = tf.layers.dense(fc1, 1024)
  fc2 = tf.reshape(fc1, [-1, w_f_2.get_shape().as_list()[0]])
  fc2 = tf.add(tf.matmul(fc2, w_f_2), b_f_2)
  fc2 = tf.nn.relu(fc2)
  # Third Fully connected layer
  # fc3 = tf.layers.dense(fc2, 1024)
  fc3 = tf.reshape(fc2, [-1, w_f_3.get_shape().as_list()[0]])
  fc3 = tf.add(tf.matmul(fc3, w_f_3), b_f_3)
  fc3 = tf.nn.relu(fc3)

  out = tf.add(tf.matmul(fc3, w_out), b_out)
  return out



In [3]:
logits = model_cnn(X, keep_prob,num_classes)
prediction = tf.nn.softmax(logits)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(loss_op)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [6]:
init = tf.global_variables_initializer()

# Please don't change these.
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2


with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # batch_x = np.reshape(batch_x, (-1, 28, 28, 1))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.8})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))


Step 1, Minibatch Loss= 20962839887872.0000, Training Accuracy= 0.195
Step 10, Minibatch Loss= 3457039204352.0000, Training Accuracy= 0.609
Step 20, Minibatch Loss= 1037471842304.0000, Training Accuracy= 0.797
Step 30, Minibatch Loss= 452625268736.0000, Training Accuracy= 0.898
Step 40, Minibatch Loss= 353807302656.0000, Training Accuracy= 0.898
Step 50, Minibatch Loss= 366857486336.0000, Training Accuracy= 0.914
Step 60, Minibatch Loss= 136697970688.0000, Training Accuracy= 0.938
Step 70, Minibatch Loss= 338091114496.0000, Training Accuracy= 0.906
Step 80, Minibatch Loss= 296592769024.0000, Training Accuracy= 0.906
Step 90, Minibatch Loss= 182943154176.0000, Training Accuracy= 0.914
Step 100, Minibatch Loss= 234992566272.0000, Training Accuracy= 0.922
Step 110, Minibatch Loss= 96217079808.0000, Training Accuracy= 0.938
Step 120, Minibatch Loss= 87716659200.0000, Training Accuracy= 0.945
Step 130, Minibatch Loss= 110613250048.0000, Training Accuracy= 0.969
Step 140, Minibatch Loss= 182